In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import random
import numpy as np
import time

In [2]:
def load_data(filename): # Cargar JSON
    data = None
    with open(filename,"r") as in_file:
        data = json.load(in_file)
    return data

In [3]:
datos = load_data('definition.json')

In [4]:
PM = json_normalize(datos['PM'])
PM

,cores,id,mainmemory,powermax,powermin,vmwareoverhead
0,2,0,1000,300,100,0.05
1,16,1,15000,1000,400,0.08
2,1,2,500,100,50,0.04
3,4,3,2000,700,150,0.06


In [5]:
VM = json_normalize(datos['VM'])
VM

,cpuusage,id,mainmemory
0,0.40,0,125
1,0.10,1,125
2,0.22,2,500
3,0.15,3,250
4,0.01,4,250
5,0.70,5,750
6,0.40,6,500


### Versión 1

In [6]:
totalW = 100000 # ¿Cómo hacer infinito?
size_VM = len(VM)
size_PM = len(PM)

contador = 0
ejecuciones = 10000
finalizador = ejecuciones * 0.05 # Finalizador 5% del nº de ejecuciones
m_asignacion = None # Variable para guardar la mejor asignación
# Fórmula del consumo de una máquina
def formula(Wmax, Wmin, UsoCPU):
    Wtotal = Wmin + (UsoCPU) * (Wmax - Wmin) 
    return Wtotal

# Consumo de una asignación
def consumo():
    total = 0
    for i in range(0,size_PM):
        if PM_Estado[0][i] != 0: # Mirar si la máquina 'i' tiene uso_cpu > 0
            actual = formula(PM.loc[i,'powermax'],PM.loc[i,'powermin'],PM_Estado[0][i]+PM.loc[i,'vmwareoverhead'])
            total += actual
    return total

# Guardar tiempo de comienzo de ejecución
start_time = time.time()
# Ejecución
for runs in range(1,ejecuciones): # 'For' 1
    VM_Random = random.sample(range(0,size_VM),size_VM) # Orden en el que se recorren las máquinas virtuales
    PM_Random = random.sample(range(0,size_PM),size_PM) # Orden en el que se recorren las máquinas físicas
    PM_Estado = [[0]*len(PM_Random), [0]*len(PM_Random)] # Para cada máquina física: 'uso_cpu' y 'uso_mem' actual
    asignacion = "\n" # Inicializar asignación
    contador += 1 # Aumentar nº de iteraciones
    # Recorrer para cada máquina virtual
    for i in VM_Random: # 'For' 2
        # Variable para saber si se ha intrucido ya la máquina virtual
        introducido = False
        # Recorrer para cada máquina física
        for j in PM_Random: # 'For' 3
            # Mirar si ya se ha introducido la máquina virtual
            if not introducido:
                # Mirar si la máquina física actual tiene suficiente cpu y mem para la máquina virtual
                if(100 - PM_Estado[0][j] - PM.loc[j,'vmwareoverhead']) >= (VM.loc[i,'cpuusage'] / PM.loc[j,'cores']) and ((PM.loc[j,'mainmemory'] - PM_Estado[1][j]) >= VM.loc[i,'mainmemory']):
                    # Introducir cpu de la máquina virtual actual en la máquina física actual
                    # CPU que necesita la máquina virtual depende del nº de cores que tiene la máquina física
                    # Ejemplo: 'cpuusage' 0.7 sobre 1 núcleo = 0.7
                    #          'cpuusage' 0.7 sobre 2 núcleo = 0.35
                    PM_Estado[0][j] += (VM.loc[i,'cpuusage'] / PM.loc[j,'cores'])
                    # Introducir mem de la máquina virtual actual en la máquina física actual
                    PM_Estado[1][j] += VM.loc[i,'mainmemory']
                    # Se ha introducido la máquina virtual
                    introducido = True
                    # Guardar asignación/introducción
                    asignacion += "VM: "+str(i)+"|PM: "+str(j)+"\n"
        # Mirar si después de recorrer todas las máquinas físicas, si NO se ha podido introducir en ninguno
        if not introducido:
            #Salir del 'for' 2 y crear otra combinación
            break
    # Calcular consumo de la asignación actual
    totalNew = consumo()
    # Mirar si es menor que la asignación anterior
    if totalW > totalNew:
        # Visualizar iteración y valor nuevo encontrado
        print("Iteración: "+str(runs)+", "+str(totalNew)+"W")
        # Guardar nueva mejor asignación
        m_asignacion = asignacion
        # Guardar nuevo valor
        totalW = totalNew
        # Resetear contador de finalización
        contador = 0
    # Mirar si el contador de finalización es igual al valor de finalización
    if contador == finalizador:
        # Salir del 'for' 1 y terminar programa
        break
# Atrapar tiempo final
end_time = time.time()
# Visualizaciones
print("\nMejor consumo energético: "+str(totalW)+"W")
print("\nAsignaciones: "+m_asignacion)
print("Ejecuciones: "+str(runs))
print("Tiempo de ejecución: "+str((end_time - start_time))+"s")

Iteración: 1, 522.25W
Iteración: 33, 472.25W
Iteración: 45, 462.625W
Iteración: 47, 450.375W

Mejor consumo energético: 450.375W

Asignaciones: 
VM: 3|PM: 2
VM: 0|PM: 2
VM: 5|PM: 3
VM: 1|PM: 2
VM: 6|PM: 3
VM: 4|PM: 3
VM: 2|PM: 3

Ejecuciones: 547
Tiempo de ejecución: 0.4970827102661133s


### Versión 2.3

In [63]:
ValorError = 1000000000 # ¿Cómo hacer infinito?
consumoMEJOR = ValorError # ¿Cómo hacer infinito?
size_VM = len(VM) # Nº de máquinas virtuales
size_PM = len(PM) # Nº de máquinas físicas
contador = 0.0
deseado = 8000
L_asignaciones = [] # Lista de asignaciones
L_asignacionesNEW = []
asig = [[0],[0],[0],[0]]    # Inicializar asignaciones (Padre, Madre, Hijo1, Hijo2)
consumo = [[0],[0],[0],[0]] # Inicializar consumos     (Padre, Madre, Hijo1, Hijo2)

iteracion = 0
# Fórmula del consumo de una máquina
def formula(Wmax, Wmin, UsoCPU):
    Wtotal = Wmin + (UsoCPU) * (Wmax - Wmin) 
    return Wtotal
# Mirar si es posible una asignación y devolver consumo total
def consumo_asignacion(h):
    PM_Estado = [[0]*size_PM, [0]*size_PM] # Estado de una máquina física ['usocpu', 'usomem']
    asignacion_es_posible = True # Variable para ver si es posible la asignación
    i_VM = 0 # Index de la posición de la lista de asignación 'h', que equivale a la máquina virtual
    for i_PM in h:
        # Mirar si el estado de la máquina física permite asignar la máquina virtual
        if(100 - PM_Estado[0][i_PM] - PM.loc[i_PM,'vmwareoverhead']) >= (VM.loc[i_VM,'cpuusage'] / PM.loc[i_PM,'cores']) and ((PM.loc[i_PM,'mainmemory'] - PM_Estado[1][i_PM]) >= VM.loc[i_VM,'mainmemory']):
            # Actualizar estado de la cpu de la máquina física
            PM_Estado[0][i_PM] += (VM.loc[i_VM,'cpuusage'] / PM.loc[i_PM,'cores'])
            # Actualizar estado de la mem de la máquina física
            PM_Estado[1][i_PM] += VM.loc[i_VM,'mainmemory']
            # Apuntar a la siguiente posición de la lista
            i_VM += 1
        else:
            # La asignación no es posible
            asignacion_es_posible = False
            # Salir del 'for'
            break
    if asignacion_es_posible:
        total = 0
        for i in range(0,size_PM):
            if PM_Estado[0][i] != 0: # Mirar si la máquina 'i' tiene uso_cpu != 0
                actual = formula(PM.loc[i,'powermax'],PM.loc[i,'powermin'],PM_Estado[0][i]+PM.loc[i,'vmwareoverhead'])
                total += actual
        return total
    else:
        return ValorError # Un consumo W infinito

start_time = time.time() # Guardar tiempo de comienzo de ejecución
while True:
    #time_medio_1 = time.time()
    if contador >= deseado: # Si durante X recorridos el consumoMEJOR no ha cambiado, hemos encontrado el deseado
        break
    finalizador = random.randint(20, 40) # Buscar entre 20 y 40 nuevas asignaciones
    while True:
        asignacion = np.random.choice(size_PM, size_VM) # Crear una lista de tamaño 'size_VM' llenando
                                                        # con números de 0 a 'size_PM'
        # Ejemplo: en la posición '0' hay un '1' -> la máquina virtual '0' se ha asignado a la máquina física '1'
        consumoNEW = consumo_asignacion(asignacion) # Obtener consumo de asignación posible o no
        if consumoNEW != ValorError: # Mirar si el consumo es diferente de Error
            # Añadir a lista de asignaciones
            L_asignaciones.append(asignacion)
            # Restar al finalizador
            finalizador -= 1
            if consumoMEJOR > consumoNEW:
                asignacionMEJOR = asignacion
                consumoMEJOR = consumoNEW
                #print("Nuevo consumo energético: "+str(consumoMEJOR)+"W, contador: "+str(contador))
                contador = 0
        contador += 1
        if finalizador == 0:
            break
    random.shuffle(L_asignaciones)
    
    for permutaciones in range(0,1): # Hacer 2 Permutaciones
        for l in range(0,len(L_asignaciones),2): # Recorrer de 2 en 2 la lista de asignaciones
            asig[0] = L_asignaciones[l]
            asig[1] = L_asignaciones[(l+1)%len(L_asignaciones)]
            # Generar permutaciones
            k = random.randint(1, size_VM - 1)
            asig[2] = np.concatenate((asig[0][:k],asig[1][k:]),axis=0) 
            asig[3] = np.concatenate((asig[1][:k],asig[0][k:]),axis=0)

            consumo[0] = consumo_asignacion(asig[0]) # Consumo Padre
            consumo[1] = consumo_asignacion(asig[1]) # Consumo Madre
            consumo[2] = consumo_asignacion(asig[2]) # Consumo Hijo1
            consumo[3] = consumo_asignacion(asig[3]) # Consumo Hijo2
            i = consumo.index(min(consumo)) # Obtener índice del consumo menor
            consumoNEW = consumo[i]
            asignacion = asig[i]
            #a = np.array(consumo) 
            #i = np.where(a == a.min()) 
            #consumoNEW = consumo[i[0][0]]
            #asignacion = asig[i[0][0]]
            if consumoNEW!=ValorError:
                L_asignacionesNEW.append(asignacion) # Coger la asignación con el menor consumo
                if consumoMEJOR > consumoNEW:
                    asignacionMEJOR = asignacion
                    consumoMEJOR = consumoNEW
                    #print("Nuevo consumo energético: "+str(consumoMEJOR)+"W, contador: "+str(contador))
                    contador = 0
                contador += 1
        L_asignaciones = L_asignacionesNEW
        random.shuffle(L_asignaciones)
        L_asignacionesNEW = [] # Reiniciar lista de asignaciones nuevas
    #time_medio_2 = time.time()
    #print("Tiempo de ejecución medio: "+str((time_medio_2 - time_medio_1))+"s")
# FIN
end_time = time.time()
print("\nMejor consumo energético: "+str(consumoMEJOR)+"W")
print("\nMejor asignación:")
i_VM = 0
for i_PM in asignacionMEJOR:
    print("VM: "+str(i_VM)+"|PM: "+str(i_PM))
    i_VM += 1

print("Tiempo de ejecución final: "+str((end_time - start_time))+"s")


Mejor consumo energético: 450.375W

Mejor asignación:
VM: 0|PM: 2
VM: 1|PM: 2
VM: 2|PM: 3
VM: 3|PM: 2
VM: 4|PM: 3
VM: 5|PM: 3
VM: 6|PM: 3
Tiempo de ejecución final: 18.42633032798767s
